In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import math

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
d_labitems = pd.read_csv('../data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

In [81]:
admissions_sample = pd.read_pickle("../data/admissions_sample_male.pkl")

In [82]:
admissions_sample

,subject_id,hadm_id,pos,ethnicity,ed_length,insurance,gender,age,pneumoia,diabetes,hypertension,atherosclero,renal_failure,arrhythmia
0,12025783,22858815,1,WHITE,2.433333,Medicare,M,78,0.0,0.0,0.0,0.0,0.0,0.0
1,12010560,22949277,1,HISPANIC/LATINO,3.766667,Other,M,53,0.0,0.0,1.0,1.0,0.0,0.0
2,11516184,27692748,1,WHITE,10.716667,Other,M,70,0.0,0.0,0.0,0.0,0.0,0.0
3,14716749,25704799,1,WHITE,5.883333,Other,M,84,0.0,0.0,0.0,0.0,0.0,0.0
4,15999159,25077928,1,BLACK/AFRICAN AMERICAN,6.833333,Other,M,71,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11247,15313582,23498737,0,WHITE,0.000000,Other,M,56,0.0,1.0,1.0,0.0,0.0,0.0
11248,12080872,28847340,0,WHITE,3.316667,Other,M,44,0.0,0.0,0.0,0.0,0.0,0.0
11249,15114996,23842511,0,WHITE,0.000000,Other,M,69,0.0,0.0,0.0,0.0,0.0,0.0
11250,12979215,26885847,0,WHITE,5.483333,Other,M,46,0.0,0.0,0.0,0.0,0.0,0.0


## Pull Chart and Lab data only where it matches our sample patient list

In [83]:
# # We have Decided not to use Chart data after exploring it further
# # However, it still comes up in our discussions, so keeping this for now
# # Will need to update this code based on other updates we've made

# %%time #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)



In [84]:
# chartevents_final.to_pickle("./data/chartevents_final.pkl")

In [85]:
# chartevents_final = pd.read_pickle("./data/chartevents_final.pkl")

In [86]:
# chartevents_final.shape

Load Lab Events table

In [87]:
# %%time

chunksize = 3*(10 ** 6)
counter=0
lab_events_sample = []
for chunk in pd.read_csv('../data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
    lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
    counter+=1
    print(f'chunk {counter} processed')

lab_events_final = pd.concat(lab_events_sample)
# 5k patients
# 3min 52s

# 10k patients
# 4min 6s

# 15k patients
# 4min 27s



chunk 1 processed
chunk 2 processed
chunk 3 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 4 processed
chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 10 processed


...

chunk 11 processed
chunk 12 processed
chunk 13 processed
chunk 14 processed
chunk 15 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 16 processed
chunk 17 processed
chunk 18 processed
chunk 19 processed
chunk 20 processed
chunk 21 processed
chunk 22 processed
chunk 23 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 24 processed
chunk 25 processed
chunk 26 processed
chunk 27 processed
chunk 28 processed
chunk 29 processed
chunk 30 processed
chunk 31 processed
chunk 32 processed
chunk 33 processed
chunk 34 processed
chunk 35 processed
chunk 36 processed
chunk 37 processed
chunk 38 processed
chunk 39 processed
chunk 40 processed
chunk 41 processed


In [88]:
lab_events_final.to_pickle("../data/lab_events_final_male.pkl")

In [8]:
# lab_events_final = pd.read_pickle("data/lab_events_final.pkl")

In [89]:
lab_events_final.shape

(6840203, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [90]:
# Can we do this filtering when generating lab data?
# For now let's not as we're discussing using different hospitalizations from the same subjects

%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(list(admissions_sample.hadm_id))]


CPU times: user 3.16 s, sys: 3.9 s, total: 7.06 s
Wall time: 17.5 s


Check how many patients have had each test done

In [91]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

itemid
51221    8355
50912    8317
50971    8303
51006    8291
51265    8288
50983    8262
50902    8249
51222    8213
51301    8213
51250    8205
Name: subject_id, dtype: int64

Most common tests performed on patients with pos diagnosis

In [92]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


itemid
50971    1407
50912    1406
50983    1402
51006    1401
50902    1400
51221    1396
51265    1391
50868    1384
50882    1384
50931    1381
Name: subject_id, dtype: int64

Item Black List

In [93]:
item_black_list = []
# # 50920 - test results are text in comments - need to come back and figure out how to handle this


Bonus Items To Add

In [94]:
# These test, although they have much missingness, are known to be good indicators of congestive heart failure

item_bonus_list = [51274, 51003, 50920, 50911, 51464]
# 51274 PT
# 50920 Estimated GFR (MDRD equation)
# 51003 Troponin T
# 50911 Creatine Kinase, MB Isoenzyme
# 51464 Specific Gravity


Take only the X most commonly performed tests

In [95]:
top_test_num = 20

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with pos diagnosis
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

itemid_sub_sample = itemid_sub_sample + item_bonus_list

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]


In [96]:
# # lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))

# top_labs = lab_events_sampled.groupby(['itemid']).agg(freq=('subject_id', lambda x: len(np.unique(x))))\
# .reset_index().sort_values('freq', ascending=False).head(60)

# top_labs


In [97]:
# subset_labs['subject_id'].reset_index(drop=True).plot(kind='line')

In [98]:
# print(lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(20).reset_index().rename(columns = {'index' : 'itemid'}))

# # print(1191 / admissions_sample[admissions_sample['pos'] == 1].subject_id.nunique())
# # print(1192 / admissions_sample[admissions_sample['pos'] == 1].subject_id.nunique())


In [99]:
# print(lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 0].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(20).reset_index().rename(columns = {'index' : 'itemid'}))

# # print(6726 / admissions_sample[admissions_sample['pos'] == 0].subject_id.nunique())
# # print(6793 / admissions_sample[admissions_sample['pos'] == 0].subject_id.nunique())


In [100]:
# subset_labs['subject_id'].reset_index(drop=True).plot(kind='line')

## Data Cleaning

Make a separate table where we run some of the forest models on the Text data?

Special updates for Urine sample tests

In [101]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150', '>150.0'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT','ERROR,DISREGARD PREVIOUS RESULT OF 11.8'])), ['value', 'valuenum']] = np.nan

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


More Data Cleaning

In [126]:
%%time

# Can probably make a helper function combining some of the below mapping

# 51466
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE', 'LRG'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51514
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', 'NORMAL.  ', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>8'])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>12.  ', '>12*.  '])), ['value', 'valuenum']] = 15

# 51464
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 3

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51487
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['POS.  ', 'POS', 'POS*.  '])), ['value', 'valuenum']] = 1

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51492
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['___'])), ['value', 'valuenum']] = np.nan
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].str.contains('TR.')), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>300'])), ['value', 'valuenum']] = 350
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>600.  ', '>600*.  '])), ['value', 'valuenum']] = 700

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51486
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' ', 'N'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51506
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Clear.  ', 'CLEAR.  ', 'Clear', ' ', 'CLEAR'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['SlHazy'])), ['value', 'valuenum']] = 0.5
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Hazy.  ', 'Hazy*.  ', 'Hazy', 'HAZY', 'HAZY*.  ', 'Slcldy', 'SLCLOUDY'])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Cloudy*.  ', 'Cloudy.  ', 'Cloudy', 'CLO', 'CLOUDY', 'CLOU', 'CLOUDY*.  '])), ['value', 'valuenum']] = 2

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51301
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51301) & (lab_events_sampled_sub['value'].isin(['<0.1'])), ['value', 'valuenum']] = 0

# 51274
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) & (lab_events_sampled_sub['value'].isin(['22.2.'])), ['value', 'valuenum']] = 22.2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) & (lab_events_sampled_sub['value'].isin(['…12.1'])), ['value', 'valuenum']] = 12.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) & (lab_events_sampled_sub['value'].isin(['>150'])), ['value', 'valuenum']] = 150

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CPU times: user 7.97 s, sys: 332 ms, total: 8.3 s
Wall time: 10.2 s


In [127]:
lab_events_sampled_sub[lab_events_sampled_sub['value']=='>150']
# itemid_sub_sample

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments


Additional Data Cleaning

In [128]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030', '.1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['comments'].isin(['>1.050*.'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>13.4'])), ['value', 'valuenum']] = 15

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.070'])), ['value', 'valuenum']] = 1.080

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) & (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = np.nan

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [129]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) 
                           & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str))
                           & ((lab_events_sampled_sub['value'].astype(str)) != round(lab_events_sampled_sub['valuenum'], 10).astype(str)), ]


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
87206156,107413387,18629932,22270452.0,51244405,51003,2168-06-04 21:19:00,2168-06-04 23:17:00,.40,0.4,ng/mL,0.0,0.01,abnormal,STAT,CTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.


## Code to look into specific test values for cleaning

In [ ]:
# lab_events_sampled_sub[lab_events_sampled_sub['value'] == 'UNABLE TO REPORT']

In [ ]:
# # Used to investigate bad data
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', 200)

# bad_id = 51274

# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id].value.value_counts().reset_index().sort_values(by = 'index')
# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['value', 'comments']].drop_duplicates()
# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['flag', 'comments', 'value', 'valuenum', 'ref_range_lower', 'ref_range_upper']].drop_duplicates().head(100)
# # lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == bad_id) & (lab_events_sampled_sub['value'].isnull()), ][['value', 'valuenum', 'comments']].drop_duplicates()


In [130]:
def gfr_grabber(patient):
    comment = patient['comments']
    gfr_vals = []
    
    if pd.isnull(comment):
        return np.nan
    if type(comment) == str:
        if comment == '___':
            return np.nan
        
        comment = comment.replace('>75', '75')
        if ('between' in comment):
            start_low = comment.find('between')
            end_low = comment.find(' and', start_low + 8, )

            end_high = comment.find(' mL', end_low + 5, )
            
            val = (int(comment[start_low + 8 : end_low]) + int(comment[end_low + 5 : end_high])) / 2

            gfr_vals = [val, val]
        elif 'likely' in comment:
            start_point = comment.find('likely')
            end_point = comment.find(' mL', start_point + 7, )
            gfr_vals = [comment[start_point + 7 : end_point], comment[start_point + 7 : end_point]]
        else:
            start_points = [i for i in range(len(comment)) if comment.startswith('GFR = ', i)]
            end_points = list(map(lambda x: comment.find(' ', x + 6, ), start_points))

            gfr_vals = list(map(lambda x, y: comment[x + 6 : y], start_points, end_points))

    if (admissions_sample.loc[admissions_sample['subject_id'] == patient['subject_id'], 'ethnicity'] == 'BLACK/AFRICAN AMERICAN').iloc[0]:
        return int(gfr_vals[1])
    else:
        return int(gfr_vals[0])
    
lab_gfr = lab_events_sampled_sub.loc[lab_events_sampled_sub['itemid'] == 50920, ['labevent_id', 'comments', 'subject_id']]
lab_gfr['value'] = lab_gfr.apply(gfr_grabber, axis = 1)


In [131]:
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'] = lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'labevent_id'].map(lambda x: lab_gfr.loc[lab_gfr['labevent_id'] == x, 'value'])
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'] = lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'].map(lambda x: x.iloc[0].astype(np.float64))
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'valuenum'] = lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'].astype(np.float64)
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'ref_range_lower'] = 60
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'ref_range_upper'] = 120



/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Get Aggregate Test Values

In [132]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


CPU times: user 464 ms, sys: 158 ms, total: 621 ms
Wall time: 821 ms


In [133]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [134]:
missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

min     1673
max     1673
mean    1673
dtype: int64

In [135]:
lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['min'].isnull(), ].itemid.value_counts()

# lab_events_sampled_sub_grouped.loc[(lab_events_sampled_sub_grouped['min'].isnull()) & (lab_events_sampled_sub_grouped['itemid'] == 51003), ]#.subject_id.nunique()
# # lab_events_sampled_sub_grouped.subject_id.nunique()

# lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['subject_id'] == 10002264, ]


51003    1396
50911     224
50920      39
51274       8
51265       4
51301       1
51277       1
Name: itemid, dtype: int64

In [136]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [137]:
lab_adm = lab_events_sampled_sub_grouped.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

lab_range_min_max_eg = lab_adm.groupby(['itemid', 'ethnicity', 'gender']).agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max_eg.reset_index(inplace = True)

lab_range_eg_dic = dict(zip(zip(lab_range_min_max_eg['itemid'], lab_range_min_max_eg['ethnicity'], lab_range_min_max_eg['gender']), zip(lab_range_min_max_eg['range_min'], lab_range_min_max_eg['range_max'])))

# # These labs have missing data for the entire ethnicity/gender combo
lab_range_eg_dic[(50911, 'AMERICAN INDIAN/ALASKA NATIVE', 'M')] = (0.0, 10.0)
lab_range_eg_dic[(51003, 'AMERICAN INDIAN/ALASKA NATIVE', 'M')] = (0.0, 0.01)

lab_range_eg_dic


{(50868, 'AMERICAN INDIAN/ALASKA NATIVE', 'M'): (8.526315789473685,
  19.473684210526315),
 (50868, 'ASIAN', 'M'): (8.441281138790035, 19.459074733096084),
 (50868, 'BLACK/AFRICAN AMERICAN', 'M'): (8.47394540942928,
  19.411910669975185),
 (50868, 'HISPANIC/LATINO', 'M'): (8.377952755905511, 19.548556430446194),
 (50868, 'OTHER', 'M'): (8.4144578313253, 19.46987951807229),
 (50868, 'UNKNOWN', 'M'): (8.586709886547812, 19.316045380875202),
 (50868, 'WHITE', 'M'): (8.417446583083171, 19.514744834893165),
 (50882, 'AMERICAN INDIAN/ALASKA NATIVE', 'M'): (22.0, 32.0),
 (50882, 'ASIAN', 'M'): (22.0, 32.0),
 (50882, 'BLACK/AFRICAN AMERICAN', 'M'): (22.0, 32.0),
 (50882, 'HISPANIC/LATINO', 'M'): (22.0, 32.0),
 (50882, 'OTHER', 'M'): (22.0, 32.0),
 (50882, 'UNKNOWN', 'M'): (22.0, 32.0),
 (50882, 'WHITE', 'M'): (22.0, 32.0),
 (50893, 'AMERICAN INDIAN/ALASKA NATIVE', 'M'): (8.400000000000002,
  10.288888888888891),
 (50893, 'ASIAN', 'M'): (8.40160000000004, 10.292400000000004),
 (50893, 'BLACK/AF

In [138]:
# # Aggregate without splitting by Ethnicity/Gender

# lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
# lab_range_min_max.reset_index(inplace = True)

# lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
# lab_range_dic


Pivot the table so we have feature columns related to test results

In [139]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [140]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


CPU times: user 154 ms, sys: 41.6 ms, total: 196 ms
Wall time: 230 ms


In [141]:
lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [142]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()#.head(40)

             itemid
abn_percent  51221      272
mean         51221      272
above_max    51221      272
max          51221      272
min          51221      272
                       ... 
             50911     6637
mean         50911     6637
             51003     7385
max          51003     7385
min          51003     7385
Length: 150, dtype: int64

## Start Data Imputation

Copy table, retain original for Random Forest

In [143]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [144]:
d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

,itemid,label,fluid,category,loinc_code
115,50868,Anion Gap,Blood,Chemistry,1863-0
212,50882,Bicarbonate,Blood,Chemistry,1963-8
222,51464,Bilirubin,Urine,Hematology,5770-3
282,50893,"Calcium, Total",Blood,Chemistry,2000-8
442,50902,Chloride,Blood,Chemistry,2075-0
511,50911,"Creatine Kinase, MB Isoenzyme",Blood,Chemistry,6773-6
512,50912,Creatinine,Blood,Chemistry,2160-0
634,50920,Estimated GFR (MDRD equation),Blood,Chemistry,NaN
723,50931,Glucose,Blood,Chemistry,6777-7
761,51221,Hematocrit,Blood,Hematology,4544-3


In [145]:
lab_events_impute

min                                                                \
itemid 50868 50882 50893  50902 50911 50912 50920  50931 50960 50970 50971   
0       13.0  24.0   9.6  104.0   NaN   0.8  75.0  108.0   1.9   3.1   4.1   
1       17.0  23.0   9.2  100.0   NaN   0.9   NaN  108.0   2.1   4.0   3.6   
2       13.0  20.0   8.9  105.0   1.0   2.5  19.0   73.0   1.8   2.0   3.3   
3       18.0  26.0   9.7   97.0   2.0   1.0  75.0  113.0   2.0   3.3   3.9   
4       17.0  22.0   9.1  109.0   NaN   0.8   NaN   93.0   2.1   3.9   4.7   
...      ...   ...   ...    ...   ...   ...   ...    ...   ...   ...   ...   
8622    11.0  29.0   9.3  103.0   1.0   0.8   NaN  101.0   2.2   3.5   3.7   
8623    14.0  27.0   8.6  101.0   NaN   1.1   NaN   98.0   2.1   3.2   4.5   
8624     9.0  24.0   8.8  102.0   NaN   0.9   NaN  121.0   1.4   3.7   4.3   
8625    12.0  18.0   8.1   89.0   NaN   0.9  65.0   92.0   1.8   2.8   3.8   
8626    11.0  26.0   9.1   99.0   NaN   0.6  75.0   87.0   2.0   4.2   3.8   

                                                                            \
itemid  50983 51003 51006 51221 51222 51248 51249 51250  51265 51274 51277   
0       141.0   NaN  18.0  39.6  13.0  31.8  32.8  97.0  257.0   NaN  13.1   
1       136.0   NaN  13.0  42.5  14.0  30.1  33.0  91.0  306.0   NaN  12.9   
2       139.0  0.02  16.0  22.3   7.2  28.3  31.2  88.0  173.0  11.8  15.5   
3       137.0   NaN   7.0  48.5  16.3  31.1  33.7  92.0  193.0  12.2  13.6   
4       143.0   NaN  10.0  27.6   8.7  29.0  30.8  93.0  228.0  11.8  14.9   
...       ...   ...   ...   ...   ...   ...   ...   ...    ...   ...   ...   
8622    139.0   NaN   9.0  40.8  14.0  31.6  34.4  92.0  231.0  11.3  12.9   
8623    137.0   NaN  18.0  38.3  13.3  29.1  34.7  84.0  282.0   NaN  13.9   
8624    142.0   NaN  10.0  29.1   9.4  29.9  32.1  93.0  184.0   NaN  13.3   
8625    119.0   NaN  13.0  19.3   6.9  28.1  33.8  81.0   47.0  13.7  16.4   
8626    140.0   NaN   7.0  30.9  10.2  27.1  32.4  83.0  273.0  12.5  14.2   

                           max                                              \
itemid 51279 51301 51464 50868 50882 50893  50902 50911 50912 50920  50931   
0       4.09   8.7   NaN  13.0  24.0   9.6  104.0   NaN   0.8  75.0  108.0   
1       4.66   9.8   NaN  17.0  23.0   9.2  100.0   NaN   0.9   NaN  108.0   
2       2.49   4.3   0.0  21.0  28.0   9.7  116.0   1.0   3.8  28.0  125.0   
3       5.25   3.4   NaN  18.0  26.0   9.7   97.0   2.0   1.0  75.0  113.0   
4       2.98   4.1   NaN  17.0  22.0   9.1  109.0   NaN   0.9   NaN   93.0   
...      ...   ...   ...   ...   ...   ...    ...   ...   ...   ...    ...   
8622    4.44   6.5   0.0  11.0  29.0   9.3  103.0   1.0   0.8   NaN  101.0   
8623    4.55  10.6   NaN  14.0  27.0   8.6  101.0   NaN   1.1   NaN   98.0   
8624    3.14   6.3   NaN  14.0  33.0   9.3  105.0   NaN   1.2   NaN  178.0   
8625    2.38   1.9   NaN  18.0  25.0   8.7  102.0   NaN   1.2  65.0  134.0   
8626    3.68   3.1   NaN  14.0  29.0  10.0  101.0   NaN   0.8  75.0   97.0   

                                                                           \
itemid 50960 50970 50971  50983 51003 51006 51221 51222 51248 51249 51250   
0        1.9   3.1   4.1  141.0   NaN  18.0  39.6  13.0  31.8  32.8  97.0   
1        2.1   4.0   3.6  136.0   NaN  13.0  42.5  14.0  30.1  33.0  91.0   
2        2.2   5.1   4.6  150.0  0.02  44.0  33.1  10.9  30.3  33.8  92.0   
3        2.0   3.3   3.9  137.0   NaN   7.0  48.5  16.3  31.1  33.7  92.0   
4        2.1   3.9   4.7  143.0   NaN  11.0  33.3  10.4  29.9  31.6  95.0   
...      ...   ...   ...    ...   ...   ...   ...   ...   ...   ...   ...   
8622     2.2   3.5   3.7  139.0   NaN   9.0  40.8  14.0  31.6  34.4  92.0   
8623     2.1   3.2   4.5  137.0   NaN  18.0  38.3  13.3  29.1  34.7  84.0   
8624     1.7   4.1   4.7  147.0   NaN  32.0  30.9  10.1  30.7  32.7  94.0   
8625     2.1   3.2   4.7  134.0   NaN  19.0  26.1   9.1  29.4  35.7  87.0   
8626     2.3   4.5   4.5  142.0   NaN  1

In [146]:
lab_events_impute = lab_events_impute.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [147]:
%%time

np.random.seed(0)

for labitem in lab_range_eg_dic:
    for ind in lab_events_impute.loc[(lab_events_impute[('mean', labitem[0])].isnull()) & (lab_events_impute['ethnicity'] == labitem[1]) & (lab_events_impute['gender'] == labitem[2])].index:
        val_max = lab_range_eg_dic[labitem][1]
        val_min = lab_range_eg_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_impute[('min', labitem[0])][ind]
        val_max = lab_events_impute[('max', labitem[0])][ind]
        val_mean = lab_events_impute[('mean', labitem[0])][ind]
        
        lab_events_impute[('min', labitem[0])][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_impute[('max', labitem[0])][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_impute[('mean', labitem[0])][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 5k patients
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

# 10k patients
# 40 items - 1m 34s

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 20.7 s, sys: 476 ms, total: 21.2 s
Wall time: 25.9 s


In [148]:
for lab_itemid in lab_range_eg_dic:
    lab_events_impute[( 'above_max', lab_itemid[0])] = lab_events_impute[( 'above_max', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute[( 'below_min', lab_itemid[0])] = lab_events_impute[( 'below_min', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute[( 'abn_percent', lab_itemid[0])] = lab_events_impute[( 'abn_percent', lab_itemid[0])].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [149]:
missing_vals = lab_events_impute.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

Series([], dtype: int64)

In [150]:
lab_events_impute_KNN = lab_events_sampled_pivot.copy()

In [151]:
lab_events_impute_KNN = lab_events_impute_KNN.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

dummy = pd.get_dummies(lab_events_impute_KNN['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
lab_events_impute_KNN = lab_events_impute_KNN.drop('ethnicity', axis=1)
lab_events_impute_KNN = pd.concat([lab_events_impute_KNN, dummy], axis=1)

dummy = pd.get_dummies(lab_events_impute_KNN['insurance'], prefix='insurance')
dummy.drop('insurance_Other', axis=1, inplace=True)
lab_events_impute_KNN = lab_events_impute_KNN.drop('insurance', axis=1)
lab_events_impute_KNN = pd.concat([lab_events_impute_KNN, dummy], axis=1)


lab_events_impute_KNN['gender'] = lab_events_impute_KNN['gender'].map({'M' : 0.0, 'F' : 1.0})
lab_events_impute_KNN['gender'] = pd.to_numeric(lab_events_impute_KNN['gender'])


In [152]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=math.trunc((lab_events_impute_KNN.shape[0] ** 0.5)))
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

# 3min 8s

CPU times: user 1min 35s, sys: 17.3 s, total: 1min 53s
Wall time: 2min 18s


In [153]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns

In [154]:
for lab_itemid in lab_range_eg_dic:
    lab_events_impute_KNN_imputed[( 'above_max', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'above_max', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'below_min', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'below_min', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid[0])].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


## Standardize the Data for use in Models

In [165]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [166]:
labs_scaled = lab_events_impute.copy()
labs_scaled_KNN = lab_events_impute_KNN_imputed.copy()
labs_scaled_missing_data = lab_events_sampled_pivot.copy()

In [167]:
labs_scaled_missing_data = labs_scaled_missing_data.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [168]:
scale_cols = ~labs_scaled.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'ethnicity', 'gender', 'insurance', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

scaler.fit(labs_scaled.loc[:, scale_cols])
labs_scaled.loc[:, scale_cols] = scaler.transform(labs_scaled.loc[:, scale_cols])

scale_cols = ~labs_scaled_KNN.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'gender', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN', 'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_UNKNOWN', 'insurance_Medicare', 'insurance_Medicaid', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

scaler.fit(labs_scaled_KNN.loc[:, scale_cols])
labs_scaled_KNN.loc[:, scale_cols] = scaler.transform(labs_scaled_KNN.loc[:, scale_cols])

scale_cols = ~labs_scaled_missing_data.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'ethnicity', 'gender', 'insurance', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

scaler.fit(labs_scaled_missing_data.loc[:, scale_cols])
labs_scaled_missing_data.loc[:, scale_cols] = scaler.transform(labs_scaled_missing_data.loc[:, scale_cols])


## Save Lab Results for Models

In [169]:
# Save non-imputed file as well for forest models

pos_final_test = labs_scaled.copy()
pos_KNN_final_test = labs_scaled_KNN.copy()
pos_final_missing_data_test = labs_scaled_missing_data.copy()


In [170]:
dummy = pd.get_dummies(pos_final_test['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
pos_final_test = pos_final_test.drop('ethnicity', axis=1)
pos_final_test = pd.concat([pos_final_test, dummy], axis=1)

pos_final_test['gender'] = pos_final_test['gender'].map({'M' : 0.0, 'F' : 1.0})
pos_final_test['gender'] = pd.to_numeric(pos_final_test['gender'])

dummy = pd.get_dummies(pos_final_test['insurance'], prefix='insurance')
dummy.drop('insurance_Other', axis=1, inplace=True)
pos_final_test = pos_final_test.drop('insurance', axis=1)
pos_final_test = pd.concat([pos_final_test, dummy], axis=1)



dummy = pd.get_dummies(pos_final_missing_data_test['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
pos_final_missing_data_test = pos_final_missing_data_test.drop('ethnicity', axis=1)
pos_final_missing_data_test = pd.concat([pos_final_missing_data_test, dummy], axis=1)

pos_final_missing_data_test['gender'] = pos_final_missing_data_test['gender'].map({'M' : 0.0, 'F' : 1.0})
pos_final_missing_data_test['gender'] = pd.to_numeric(pos_final_missing_data_test['gender'])

dummy = pd.get_dummies(pos_final_missing_data_test['insurance'], prefix='insurance')
dummy.drop('insurance_Other', axis=1, inplace=True)
pos_final_missing_data_test = pos_final_missing_data_test.drop('insurance', axis=1)
pos_final_missing_data_test = pd.concat([pos_final_missing_data_test, dummy], axis=1)


In [171]:
labitem_names = d_labitems.loc[d_labitems['itemid'].isin(itemid_sub_sample)].copy()
labitem_names['Short Name'] = labitem_names['label'] + ' - ' + labitem_names['fluid'] + ' - ' + labitem_names['category']

labitem_names = labitem_names[['itemid', 'Short Name']]
labitem_names.set_index('itemid', inplace = True)
labitem_dict = labitem_names.to_dict()['Short Name']


In [172]:
def rename_lab_col(table_name):
    num_cols = ~table_name.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'gender', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN', 'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_UNKNOWN', 'age','ed_length', 'insurance_Medicare','insurance_Medicaid', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

    col_names = []
    col_names = list(table_name.loc[:, num_cols].columns)

    x, y = zip(*col_names)
    col_names = list(zip(x, list(map(labitem_dict.get, y))))

    rename_dict = dict(zip(table_name.loc[:, num_cols].columns, col_names))

    table_name.rename(columns = rename_dict, inplace = True)


In [173]:
rename_lab_col(pos_final_test)
rename_lab_col(pos_KNN_final_test)
rename_lab_col(pos_final_missing_data_test)

In [174]:
pos_final_test.to_csv('../data/pos_final_test_male.csv.gz', compression="gzip")
pos_KNN_final_test.to_csv('../data/pos_KNN_final_test_male.csv.gz', compression="gzip")
pos_final_missing_data_test.to_csv('../data/pos_final_missing_data_test_male.csv.gz', compression="gzip")

## Check for Multicollinearity

In [175]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [176]:
# %%time

# num_cols = ~pos_final_test.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos'])

# labs_scaled_features = pos_final_test.loc[:, num_cols]
# add_constant(labs_scaled_features)

# vif_vals = [variance_inflation_factor(labs_scaled_features.values, i) for i in range(labs_scaled_features.shape[1])]
# vif_df = pd.DataFrame(vif_vals, index=labs_scaled_features.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)


In [177]:
# %%time

# cols_to_drop = []
# cols_to_use = ~labs_scaled_features.columns.isin(cols_to_drop)
# labs_scaled_features_limited = labs_scaled_features.loc[:, cols_to_use]

# while (vif_df.iloc[0, 1] > 5):
#     vif_df = pd.DataFrame(vif_vals, index=labs_scaled_features_limited.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
#     cols_to_drop.append(vif_df.iloc[0, 0])
#     cols_to_use = ~labs_scaled_features.columns.isin(cols_to_drop)
    
#     labs_scaled_features_limited = labs_scaled_features.loc[:, cols_to_use]
    
#     vif_vals = [variance_inflation_factor(labs_scaled_features_limited.values, i) for i in range(labs_scaled_features_limited.shape[1])]
#     vif_df = pd.DataFrame(vif_vals, index=labs_scaled_features_limited.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
# cols_to_drop  

In [178]:
%%time

num_cols_KNN = ~pos_KNN_final_test.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos'])

labs_scaled_features_KNN = pos_KNN_final_test.loc[:, num_cols_KNN]
add_constant(labs_scaled_features_KNN)

vif_vals_KNN = [variance_inflation_factor(labs_scaled_features_KNN.values, i) for i in range(labs_scaled_features_KNN.shape[1])]
vif_df_KNN = pd.DataFrame(vif_vals_KNN, index=labs_scaled_features_KNN.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


CPU times: user 2min 17s, sys: 7.89 s, total: 2min 25s
Wall time: 1min 49s


In [179]:
%%time

cols_to_drop_KNN = []
cols_to_use_KNN = ~labs_scaled_features_KNN.columns.isin(cols_to_drop_KNN)
labs_scaled_features_limited_KNN = labs_scaled_features_KNN.loc[:, cols_to_use_KNN]

while (vif_df_KNN.iloc[0, 1] > 5):
    vif_df_KNN = pd.DataFrame(vif_vals_KNN, index=labs_scaled_features_limited_KNN.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
    cols_to_drop_KNN.append(vif_df_KNN.iloc[0, 0])
    cols_to_use_KNN = ~labs_scaled_features_KNN.columns.isin(cols_to_drop_KNN)
    
    labs_scaled_features_limited_KNN = labs_scaled_features_KNN.loc[:, cols_to_use_KNN]
    
    vif_vals_KNN = [variance_inflation_factor(labs_scaled_features_limited_KNN.values, i) for i in range(labs_scaled_features_limited_KNN.shape[1])]
    vif_df_KNN = pd.DataFrame(vif_vals_KNN, index=labs_scaled_features_limited_KNN.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
cols_to_drop_KNN  


CPU times: user 1h 44min 48s, sys: 5min 18s, total: 1h 50min 7s
Wall time: 1h 19min 24s


[('max', 'Bilirubin - Urine - Hematology'),
 ('mean', 'Red Blood Cells - Blood - Hematology'),
 ('mean', 'MCH - Blood - Hematology'),
 ('max', 'Glucose - Blood - Chemistry'),
 ('mean', 'Hemoglobin - Blood - Hematology'),
 ('max', 'PT - Blood - Hematology'),
 ('mean', 'Estimated GFR (MDRD equation) - Blood - Chemistry'),
 ('min', 'Hemoglobin - Blood - Hematology'),
 ('mean', 'MCV - Blood - Hematology'),
 ('mean', 'RDW - Blood - Hematology'),
 ('max', 'Creatinine - Blood - Chemistry'),
 ('min', 'Hematocrit - Blood - Hematology'),
 ('mean', 'Bilirubin - Urine - Hematology'),
 ('mean', 'MCHC - Blood - Hematology'),
 ('mean', 'Creatinine - Blood - Chemistry'),
 ('max', 'Red Blood Cells - Blood - Hematology'),
 ('mean', 'Troponin T - Blood - Chemistry'),
 ('max', 'MCH - Blood - Hematology'),
 ('below_min', 'Hemoglobin - Blood - Hematology'),
 ('max', 'Creatine Kinase, MB Isoenzyme - Blood - Chemistry'),
 ('min', 'MCH - Blood - Hematology'),
 ('max', 'Urea Nitrogen - Blood - Chemistry'),
 ('m

In [180]:
# pos_final_test_trimmed = labs_scaled_features.loc[:, cols_to_use].merge(pos_final_test[['subject_id', 'hadm_id', 'pos']], left_index = True, right_index = True)
pos_KNN_final_test_trimmed = labs_scaled_features_KNN.loc[:, cols_to_use_KNN].merge(pos_KNN_final_test[['subject_id', 'hadm_id', 'pos']], left_index = True, right_index = True)

In [181]:
# pos_final_test_trimmed.to_csv('../data/pos_final_test_trimmed_male.csv.gz', compression="gzip")
pos_KNN_final_test_trimmed.to_csv('../data/pos_KNN_final_test_trimmed_male.csv.gz', compression="gzip")